# Solving a complex task with a multi-agent hierarchy


In [ ]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 8.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [ ]:
!pip install smolagents[telemetry] opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

In [ ]:
import base64
import os

os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""

LANGFUSE_PUBLIC_KEY = os.environ.get("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.environ.get("LANGFUSE_SECRET_KEY")

if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("Langfuse public or secret keys are missing!")

LANGFUSE_AUTH = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

print("Langfuse environment variables set successfully!")

Langfuse environment variables set successfully!


In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

In [ ]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool


model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

We can start with creating a baseline, simple agent to give us a simple report.

In [ ]:
task = """Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations"""

In [ ]:
from google.colab import userdata
import os
os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [ ]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.                      │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/smolagents/models.py:393: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/smolagents/models.py:402: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  film_locations_result = web_search(query="List of Harry Potter filming locations", filter_year=None)             
  print(film_locations_result)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [Harry Potter filming locations guide - VisitBritain 
Shop](https://www.visitbritainshop.com/us/en/harry-potter-filming-locations-guide)
Source: visitbritainshop.com

You can find Harry Potter filming locations all over the United Kingdom – from the wild Scottish Highlands home of 
Hagrid's Hut to the historic London market ...

1. [Top Sights Where 'Harry Potter' Was 
Filmed](https://www.ricksteves.com/watch-read-listen/read/articles/harry-potter-sites)
Source: Rick Steves

Top Sights Where 'Harry Potter' Was Filmed · London · Near Bath · Near Brighton · Oxford · Durham and Northeast 
England · Scottish Highlands · Ireland.

2. [31 Must Visit Harry Potter Filming Locations in the UK and 
...](https://flavourofthefilm.com/2024/11/01/31-must-visit-harry-potter-filming-locations-in-the-uk-and-ireland/)
Date published: Nov 1, 2024
Source: Flavour of the Film

Quick shots of London in the films also include: the Thames River, Canary Wharf, Houses of Parliament, Tower Bridge
and the museum ship HMS ...

3. [Places in Harry Potter](https://en.wikipedia.org/wiki/Places_in_Harry_Potter)
Source: Wikipedia

1.1 Number 4, Privet Drive · 1.2 Godric's Hollow · 1.3 The Burrow · 1.4 Little Hangleton · 1.5 Malfoy Manor · 1.6 
Number 12, Grimmauld Place · 1.7 Spinner's End · 1.8 ...

4. [The Complete Guide to Harry Potter Film Locations ...](https://londonmymind.com/harry-potter-film-locations/)
Date published: May 9, 2023
Source: londonmymind.com

Sure, the studios in Leavesden were one of the primary Harry Potter film locations, but they also filmed across 
London, England, the U.K., and ...

5. [UK Harry Potter Filming Locations & 
Attractions](https://www.visitbritain.com/en/things-to-do/visit-harry-potter-filming-locations)
Source: VisitBritain

Harry Potter film locations around Britain · Freshwater West, Pembrokeshire, Wales · Lacock Abbey, Wiltshire, 
England · Alnwick Castle, Northumberland, England.

6. [A Magical Guide to the Best Harry Potter Places to 
Visit](https://heleneinbetween.com/2025/03/harry-potter-filming-locations-best-places-to-travel-for-potterheads.htm
l)
Source: Helene in Between

If you're a Harry Potter fan, you have to visit the Warner Brothers Studio in London. It's the place where all 8 of
the Harry Potter movies were filmed. You can ...

7. [I explored EVERY Harry Potter filming 
location](https://www.youtube.com/watch?v=Tbx-AFBuDY0&pp=0gcJCfwAo7VqN5tD)
Source: YouTube · Be Reyt

I set out on a journey to visit *all 60+* Harry Potter filming locations and in this video you can join in my 
adventure.

8. [Where was Harry Potter filmed? Top filming 
locations](https://www.trafalgar.com/real-word/harry-potter-filming-locations/)
Source: Trafalgar

1. Alnwick Castle, Northumberland · 2. Christ Church College, Oxford · 3. Gloucester Cathedral, Gloucestershire · 
4. Lacock Abbey, Wiltshire · 5. Durham Cathedral, ...

9. [32 Places Every Harry Potter Fan Needs to Visit in Their 
...](https://www.housebeautiful.com/lifestyle/g3850/harry-potter-attractions/)
Date published: Mar 9, 2023
Source: House Beautiful

Freshwater West Beach in Pembrokeshire, Wales is where Dobby's death was filmed, and there's a real-life memorial 
for the beloved house-elf.

Out: None

...

[Step 7: Duration 16.10 seconds| Input tokens: 103,565 | Output tokens: 3,411]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Manually specify nearby cricket stadiums based on the latest web search results                                
  nearby_stadiums_manual = {                                                                                       
      "Leavesden Studios": "Abbots Langley Cricket Club, Everett Rovers, Watford Leisure Centre Woodside",         
      "Hogwarts Express station": "Lord's Cricket Ground",                                                         
      "Edinburgh Castle": "Grange Cricket Club",                                                                   
      "Gloucester Cathedral": "Seat Unique Stadium",                                                               
      "Lacock Abbey": "Lacock Rec",                                                                                
      "Alnwick Castle": "Alnwick Cricket Club",                                                                    
      "Durham Cathedral": "Emirates Riverside, Durham University Ground",                                          
      "Hampton Court Palace": "Hampton Court Green, East Molesey Cricket Club",                                    
      "Elstree Borehamwood Studios": "Lord's Cricket Ground",                                                      
      "London Film Studios": "Lord's Cricket Ground, Queen's Cricket Club"                                         
  }                                                                                                                
                                                                                                                   
  # Combine the results into a pandas DataFrame                                                                    
  import pandas as pd                                                                                              
                                                                                                                   
  df_film_locations = pd.DataFrame({                                                                               
      "Location": locations.keys(),                                                                                
      "Latitude": [coord[0] for coord in locations.values()],                                                      
      "Longitude": [coord[1] for coord in locations.values()],                                                     
      "Travel Time to Pune": time_to_locations.values(),                                                           
      "Nearby Cricket Stadiums": nearby_stadiums_manual.values()                                                   
  })                                                                                                               
                                                                                                                   
  final_answer(df_film_locations)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                       Location   Latitude  Longitude Travel Time to Pune  \
0            Leavesden Studios  51.690144  -0.418115             9:03:36   
1     Hogwarts Express station  51.536600  -0.141140             9:02:24   
2             Edinburgh Castle  55.948610  -3.200830             9:15:36   
3         Gloucester Cathedral  51.867590  -2.246770             9:12:00   
4                 Lacock Abbey  51.414750  -2.117180             9:11:24   
5               Alnwick Castle  55.415583  -1.705920             9:09:36   
6             Durham Cathedral  54.773500  -1.575770             9:09:00   
7         Hampton Court Palace  51.403330  -0.337500             9:03:00   
8  Elstree Borehamwood Studios  51.658190  -0.269030             9:03:00   
9          London Film Studios  51.552000  -0.097000             9:01:48   

                             Nearby Cricket Stadiums  
0  Abbots Langley Cricket Club, Everett Rovers, W...  
1                              Lord's Cricket Ground  
2                                Grange Cricket Club  
3                                Seat Unique Stadium  
4                                         Lacock Rec  
5                               Alnwick Cricket Club  
6       Emirates Riverside, Durham University Ground  
7     Hampton Court Green, East Molesey Cricket Club  
8                              Lord's Cricket Ground  
9        Lord's Cricket Ground, Queen's Cricket Club  

[Step 8: Duration 6.86 seconds| Input tokens: 147,716 | Output tokens: 3,766]

In [ ]:
result

,Location,Latitude,Longitude,Travel Time to Pune,Nearby Cricket Stadiums
0,Leavesden Studios,51.690144,-0.418115,9:03:36,"Abbots Langley Cricket Club, Everett Rovers, W..."
1,Hogwarts Express station,51.536600,-0.141140,9:02:24,Lord's Cricket Ground
2,Edinburgh Castle,55.948610,-3.200830,9:15:36,Grange Cricket Club
3,Gloucester Cathedral,51.867590,-2.246770,9:12:00,Seat Unique Stadium
4,Lacock Abbey,51.414750,-2.117180,9:11:24,Lacock Rec
5,Alnwick Castle,55.415583,-1.705920,9:09:36,Alnwick Cricket Club
6,Durham Cathedral,54.773500,-1.575770,9:09:00,"Emirates Riverside, Durham University Ground"
7,Hampton Court Palace,51.403330,-0.337500,9:03:00,"Hampton Court Green, East Molesey Cricket Club"
8,Elstree Borehamwood Studios,51.658190,-0.269030,9:03:00,Lord's Cricket Ground
9,London Film Studios,51.552000,-0.097000,9:01:48,"Lord's Cricket Ground, Queen's Cricket Club"


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to     │
│ here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.                      │
│ Also give me some cricket stadiums if there are any nearby to the filming locations                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- We are in Pune, with coordinates: 18.5204° N, 73.8567° E.
- We need to find all Harry Potter filming locations worldwide.
- We need to calculate the travel time to these locations from Pune via passenger plane.
- We need to find nearby cricket stadiums to these filming locations.

### 1.2. Facts to look up
- List of all Harry Potter filming locations worldwide.
  - Source: Wikipedia or official Harry Potter production websites.
- Coordinates of each Harry Potter filming location.
  - Source: Wikipedia or official Harry Potter production websites.
- Coordinates of nearby cricket stadiums to each Harry Potter filming location.
  - Source: Wikipedia or official cricket stadium websites.
- Average cruising speed of a passenger plane.
  - Source: Aviation industry standards or official airline websites.

### 1.3. Facts to derive
- Travel time from Pune to each Harry Potter filming location.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of Pune and each filming 
location, and the average cruising speed of a passenger plane.

## 2. Plan
1. Perform a web search to find a comprehensive list of all Harry Potter filming locations worldwide.
2. Visit the source URLs of the search results to confirm and gather the list of filming locations.
3. For each filming location, perform a web search to find its coordinates.
4. Visit the source URLs of the search results to confirm and gather the coordinates of each filming location.
5. For each filming location, perform a web search to find nearby cricket stadiums and their coordinates.
6. Visit the source URLs of the search results to confirm and gather the coordinates of nearby cricket stadiums.
7. Perform a web search to find the average cruising speed of a passenger plane.
8. Visit the source URL of the search result to confirm the average cruising speed of a passenger plane.
9. Use the `calculate_cargo_travel_time` function to calculate the travel time from Pune to each Harry Potter 
filming location.
10. Compile the data into a pandas dataframe, including the filming location, its coordinates, travel time from 
Pune, nearby cricket stadiums, and their coordinates.
11. Return the compiled dataframe as the final answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Perform a web search to find a comprehensive list of all Harry Potter filming locations worldwide.     
  hp_filming_locations_search = web_search(query="Harry Potter filming locations")                                 
  print(hp_filming_locations_search)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [Harry Potter filming locations guide - VisitBritain 
Shop](https://www.visitbritainshop.com/us/en/harry-potter-filming-locations-guide)
Source: visitbritainshop.com

Where was Harry Potter filmed in England? · Durham Cathedral · Alnwick Castle · Malham Cove, Skipton · New College,
Oxford · Lacock Abbey · Virginia Water.

1. [UK Harry Potter Filming Locations & 
Attractions](https://www.visitbritain.com/en/things-to-do/visit-harry-potter-filming-locations)
Source: VisitBritain

Harry Potter film locations around Britain · Freshwater West, Pembrokeshire, Wales · Lacock Abbey, Wiltshire, 
England · Alnwick Castle, Northumberland, England.

2. [Top Sights Where 'Harry Potter' Was 
Filmed](https://www.ricksteves.com/watch-read-listen/read/articles/harry-potter-sites)
Source: Rick Steves

Scottish Highlands. Many of the movies' exterior shots — especially scenes of the Hogwarts grounds — were filmed in
the craggy, cloudy Highlands of Scotland ( ...

3. [31 Must Visit Harry Potter Filming Locations in the UK and 
...](https://flavourofthefilm.com/2024/11/01/31-must-visit-harry-potter-filming-locations-in-the-uk-and-ireland/)
Date published: Nov 1, 2024
Source: Flavour of the Film

Harry Potter Filming Locations: London · The Reptile House, London Zoo · Entrance to The Leaky Cauldron · Platform 
9 ¾, King's Cross Station.

4. [Harry Potter Filming Locations in England](https://www.authenticvacations.com/harry-potter-filming-locations/)
Source: Authentic Vacations

Explore Hogwarts, Little Whinging, and more with this guide to Harry Potter Filming locations in England for the 
serious Harry Potter fan.

5. [I explored EVERY Harry Potter filming location](https://www.youtube.com/watch?v=Tbx-AFBuDY0)
Source: YouTube · Be Reyt

I set out on a journey to visit *all 60+* Harry Potter filming locations and in this video you can join in my 
adventure.

6. [Places in Harry Potter](https://en.wikipedia.org/wiki/Places_in_Harry_Potter)
Source: Wikipedia

The Leaky Cauldron · Diagon Alley · Knockturn Alley · Platform Nine and Three-Quarters, King's Cross · Hogsmeade · 
La Place Cachée, Paris.

7. [Top 7 Harry Potter Filming Locations 
Blog](https://www.scottishtours.co.uk/blog/top-7-harry-potter-filming-locations-in-scotland-and-england/)
Source: Scottish Tours

Lacock Village and Abbey​​ This 13th-century abbey in Wiltshire and the pretty nearby village were used extensively
for Harry Potter filming locations in England.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  import re                                                                                                        
                                                                                                                   
  # Step 1: Compile a comprehensive list of Harry Potter filming locations.                                        
  hp_filming_locations = [                                                                                         
      "Durham Cathedral",                                                                                          
      "Alnwick Castle",                                                                                            
      "Malham Cove, Skipton",                                                                                      
      "New College, Oxford",                                                                                       
      "Lacock Abbey",                                                                                              
      "Virginia Water",                                                                                            
      "Freshwater West, Pembrokeshire, Wales",                                                                     
      "Alnwick Castle, Northumberland, England",                                                                   
      "Scottish Highlands",                                                                                        
      "London",                                                                                                    
      "The Reptile House, London Zoo",                                                                             
      "Entrance to The Leaky Cauldron",                                                                            
      "Platform 9 ¾, King's Cross Station",                                                                        
      "Hogsmeade",                                                                                                 
      "La Place Cachée, Paris"                                                                                     
  ]                                                                                                                
                                                                                                                   
  # Step 2: Search for and confirm the coordinates of each filming location.                                       
  coordinates = {}                                                                                                 
  for location in hp_filming_locations:                                                                            
      search_query = f"{location} coordinates"                                                                     
      coordinates[location] = web_search(query=search_query)                                                       
      print(f"Coordinates for {location}: {coordinates[location]}")                                                
                                                                                                                   
  # Step 3: Search for nearby cricket stadiums for each filming location.                                          
  nearby_cricket_stadiums = {}                                                                                     
  for location in hp_filming_locations:                                                                            
      search_query = f"cricket stadiums near {location}"                                                           
      nearby_cricket_stadiums[location] = web_search(quer

Execution logs:
Coordinates for Durham Cathedral: ## Search Results
0. [Latitude and longitude of Durham 
Cathedral](https://latitude.to/articles-by-country/gb/united-kingdom/4583/durham-cathedral)
Source: latitude.to

A cathedral in the city of Durham, England, the seat of the Anglican Bishop of Durham. Latitude: 54° 46' 14.99" N 
Longitude: -1° 34' 20.39" W

1. [Durham Cathedral](https://en.wikipedia.org/wiki/Durham_Cathedral)
Source: Wikipedia

Durham Cathedral. Location within Durham. 54°46′25″N 1°34′34″W﻿ / ﻿54.77361°N 1.57611°W﻿ / 54.77361; -1.57611. 
Location, Durham. Country, England.

2. [File:Durham cathedral (geograph 
6974622).jpg](https://commons.wikimedia.org/wiki/File:Durham_cathedral_(geograph_6974622).jpg)
Date published: Jun 1, 2022
Source: Wikimedia

Wikimedia maps | Map data © OpenStreetMap contributors · heading : 225 degree. coordinates of depicted place. Edit.
54°46'25.03"N, 1°34'35.51"W.

3. [Durham Cathedral Exterior, Durham, United 
Kingdom](https://www.washingtonruralheritage.org/digital/collection/p17176coll1/id/15/)
Source: Washington Rural Heritage

Location (coordinates), 46.9989,-120.53039 ; Date, 1093 - 1133 ; Notes, Information from: 
https://en.wikipedia.org/wiki/Durham_Cathedral ; Latitude, -120.53039.

4. [Durham Cathedral](https://www.findagrave.com/cemetery/658459/durham-cathedral)
Source: Find a Grave

Get directions Durham, County Durham, England. Coordinates: 54.77311, -1.57624 · Cemetery ID: 658459 ...

5. [Durham Cathedral, Durham ::: Visit a 
Church](https://www.visit-a-church.info/sacred-buildings/church/detail/25222544059532148?lang=en)
Source: Visit a Church

Geogr. Coordinates: 54.7735° N, 1.57577° W · Geo Location ; Reference year: 1093 ; Architectural style: Romanesque 
; Building type: Basilica ; Description: ...

6. [Durham Cathedral, Durham, England DH1 3EH, GB](https://www.mapquest.com/gb/england/durham-cathedral-513883353)
Source: MapQuest

Get more information for Durham Cathedral in Durham, England. See reviews, map, get the address, and find 
directions.

7. [Durham Cathedral](http://wikimapia.org/6303/Durham-Cathedral)
Source: Wikimapia

Durham Cathedral Durham DH1 3EH, United Kingdom http://www.durhamcathedral ... Coordinates: 54°46'23"N 1°34'35"W. 
Add your comment in english. Comments.

8. [Latitude and longitude of Durham Cathedral 
Priory](https://latitude.to/articles-by-country/gb/united-kingdom/155698/durham-cathedral-priory)
Source: latitude.to

GPS coordinates of Durham Cathedral Priory, United Kingdom. Latitude: 54.7732 Longitude: -1.5768.

9. [. Durham Cathedral, Durham, UK – aerial photo](https://www.cambridgeairphotos.com/location/35ko023/)
Source: Cambridge air photos

Latitude: 54.773083. Longitude: -1.577154. Northings: 542100. Eastings: 427300. OS map sheet: n/a. Copyright: CUCAP
. Durham Cathedral, Durham, UK. Thumbnail.
Coordinates for Alnwick Castle: ## Search Results
0. [Alnwick Castle](https://en.wikipedia.org/wiki/Alnwick_Castle)
Source: Wikipedia

Alnwick Castle ; 55°24′57″N 1°42′22″W﻿ / ﻿55.4158°N 1.7062°W﻿ / 55.4158; -1.7062 · 11th century.

1. [Coordinates Alnwick-Castle-Alnwick-Northumberland- 
...](https://coordinates-converter.com/en/search/Alnwick-Castle-Alnwick-Northumberland-NE66-1NQ-Vereinigtes-Koenigr
eich)
Source: Online converter to all coordinate systems

The northernmost point is about 56 degrees and therefore the maximum value for H is 6200000. The southernmost point
is about 46 degrees and therefore the ...

2. [Alnwick Castle / Hogwarts Castle - Harry Potter 
Location](https://www.silverscreen.tours/location/alnwick-castle-hogwarts-castle/)
Source: Silverscreen Tours

Alnwick Castle / Hogwarts Castle · Harry Potter Tours · 55.415583, -1.705920 · Alnwick NE66 1NQ, England, United 
Kingdom. Location map (click to zoom). Click to ...

3. [Alnwick Castle Postcode Map.](https://www.postcode-checker.co.uk/postcode/Alnwick-Castle)
Source: Postcode-checker.co.uk

Alnwick Castle is located in the NE66 postcode (Northumberland, geo coor

[Step 5: Duration 24.19 seconds| Input tokens: 38,474 | Output tokens: 2,409]

                                   Location Coordinates  \
0                          Durham Cathedral        None   
1                            Alnwick Castle        None   
2                      Malham Cove, Skipton        None   
3                       New College, Oxford        None   
4                              Lacock Abbey        None   
5                            Virginia Water        None   
6     Freshwater West, Pembrokeshire, Wales        None   
7   Alnwick Castle, Northumberland, England        None   
8                        Scottish Highlands        None   
9                                    London        None   
10            The Reptile House, London Zoo        None   
11           Entrance to The Leaky Cauldron        None   
12       Platform 9 ¾, King's Cross Station        None   
13                                Hogsmeade        None   
14                   La Place Cachée, Paris        None   

   Travel Time to Pune (hours)  \
0                    

In [ ]:
detailed_report

,Location,Coordinates,Travel Time to Pune (hours),Nearby Cricket Stadiums
0,Durham Cathedral,None,None,## Search Results\n0. [Cricket Ground in Durha...
1,Alnwick Castle,None,None,## Search Results\n0. [ground details - Northu...
2,"Malham Cove, Skipton",None,None,## Search Results\n0. [Map of Yorkshire Cricke...
3,"New College, Oxford",None,None,## Search Results\n0. [New College Ground](htt...
4,Lacock Abbey,None,None,## Search Results\n0. [Cricket Grounds in Engl...
5,Virginia Water,None,None,## Search Results\n0. [Blackwater Cricket Asso...
6,"Freshwater West, Pembrokeshire, Wales",None,None,## Search Results\n0. [List of cricket grounds...
7,"Alnwick Castle, Northumberland, England",None,None,## Search Results\n0. [ground details - Northu...
8,Scottish Highlands,None,None,## Search Results\n0. [Cricket Grounds in Scot...
9,London,None,None,## Search Results\n0. [Cricket grounds in Lond...


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
os.environ["SERPER_API_KEY"] = "" # Changed to SERPER_API_KEY and getting from userdata

model = InferenceClientModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(

model=OpenAIServerModel("gpt-4o", max_tokens=8096),
   tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
    "geopandas",
    "plotly",
    "plotly.io",  # explicitly add this
    "plotly.express",  # explicitly add this too
    "shapely",
    "json",
    "pandas",
    "numpy",
    "kaleido"
],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

CodeAgent | gpt-4o
├── ✅ Authorized imports: ['geopandas', 'plotly', 'plotly.io', 'plotly.express', 'shapely', 'json', 'pandas', 
│   'numpy', 'kaleido']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │               

In [ ]:
!pip install markdownify requests
!pip install -U kaleido


manager_agent.run("""
ind all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.
Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ind all Harry Potter filming locations in the world, calculate the time to transfer via passenger plane to here │
│ (we're in Pune, 18.5204° N, 73.8567° E), and return them to me as a pandas dataframe.                           │
│ Also give me some cricket stadiums if there are any nearby to the filming locations. You need at least 6 points │
│ in total.                                                                                                       │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
manager_agent.python_executor.state["fig"]